In [1]:
from dotenv import load_dotenv
from together import Together
import os
import json
load_dotenv()
client = Together()

In [12]:
prompt = """
You are the operator of a runaway trolley hurtling down a track. Ahead, five workers are tied to the track and unable to move. If you do nothing, the trolley will kill all five workers. However, you notice a lever that can divert the trolley onto a different track. On this second track, there is one worker tied down who would be killed instead.
"""

In [13]:
def ethical_choice(fw):
    if fw == "d":
        res = "Prioritize deontological ethics and tell the course of action."
    elif fw == "u":
        res = "Prioritize utilitarian ethics and tell the course of action."
    elif fw == "v":
        res = "Prioritize virtue ethics and tell the course of action."
    else:
        res = "Tell the course of action"
    
    return res

In [18]:
response = client.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    messages=[{"role": "user", "content": prompt+ ethical_choice("d") +""" Always respond in JSON as \{"response": model_response\} where model_response is the response from model"""}],
)

In [19]:
json.loads(response.choices[0].message.content.strip())

{'response': 'From a deontological ethics perspective, which emphasizes the importance of moral rules and duties, the primary consideration is the moral principle of not actively causing harm or taking an action that results in the death of another person. In this scenario, diverting the trolley onto the second track would involve actively causing the death of one worker, even though it would save the lives of the five workers on the original track. According to deontological ethics, particularly as argued by philosophers like Immanuel Kant, it is morally wrong to use a person as a means to an end, even if the end is a good one. Therefore, the deontological approach would suggest that you should not pull the lever, as doing so would involve actively causing the death of one person, and instead, you should not interfere, allowing the events to unfold without your direct intervention, even though this leads to the death of the five workers. This stance prioritizes the adherence to moral 